<a href="https://colab.research.google.com/github/nourhanekefsi/RISCV_singleCycle/blob/master/Code%20parrallel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!python --version
!nvcc --version

# Installer l'extension CUDA pour Jupyter
!pip install nvcc4jupyter
%load_ext nvcc4jupyter


Python 3.10.12
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmpqv9ossnt".


In [9]:
%%cuda
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#define STB_IMAGE_IMPLEMENTATION
#define STB_IMAGE_WRITE_IMPLEMENTATION
#include "/content/stb_image.h"
#include "/content/stb_image_write.h"

__global__ void thresholding(unsigned char *image, int seuil, int n, unsigned char *imageResult) {
    int idx = threadIdx.x + blockDim.x * blockIdx.x;
    if (idx < n) {
        imageResult[idx] = imageResult[idx] = (image[idx] / 128) * 255;
    }
}

int main() {
    clock_t start_total, end_total;
    start_total = clock();

    int width, height, channels;

    unsigned char *image = stbi_load("/content/images.png", &width, &height, &channels, 1);

    if (image == NULL) {
        printf("Erreur lors du chargement de l'image\n");
        return 1;
    }

    int n = width * height;

    unsigned char *h_imageResult = (unsigned char*)malloc(n * sizeof(unsigned char));

    unsigned char *d_image, *d_imageResult;
    cudaMalloc((void**)&d_image, n * sizeof(unsigned char));
    cudaMalloc((void**)&d_imageResult, n * sizeof(unsigned char));

    cudaMemcpy(d_image, image, n * sizeof(unsigned char), cudaMemcpyHostToDevice);

    int seuil = 127;

    int blockSize = 64;
    int numBlocks = (n + blockSize - 1) / blockSize;

    thresholding<<<numBlocks, blockSize>>>(d_image, seuil, n, d_imageResult);

    cudaDeviceSynchronize();

    cudaMemcpy(h_imageResult, d_imageResult, n * sizeof(unsigned char), cudaMemcpyDeviceToHost);
    end_total = clock();
    double total_time = ((double)(end_total - start_total)) / CLOCKS_PER_SEC;
    printf("Temps total d'exécution : %.3f secondes\n", total_time);

    stbi_write_png("thresholded_image.png", width, height, 1, h_imageResult, width);

    stbi_image_free(image);
    free(h_imageResult);
    cudaFree(d_image);
    cudaFree(d_imageResult);

    return 0;
}


Temps total d'exécution : 0.202 secondes



code parralle de segmentation en utilisant otsu

In [ ]:
%%cuda
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#define STB_IMAGE_IMPLEMENTATION
#define STB_IMAGE_WRITE_IMPLEMENTATION
#include "/content/stb_image.h"
#include "/content/stb_image_write.h"
#include <math.h>

__constant__ int N;
__constant__ int somme;

__global__ void otsu_segmentation(unsigned char *image, int n, unsigned char *imageResult, int *histogram, float *moyennes, float *weights, float *caracteristiques) {
    int idx = threadIdx.x + blockDim.x * blockIdx.x;

    // Calcul de l'histogramme
    if (idx < n) {
        atomicAdd(&histogram[image[idx]], 1);
    }
    __syncthreads();

    int tid = threadIdx.x;
    int T = blockIdx.x;
    float sum = 0.0f;
    __shared__ int shared_sum[256];

    if (tid < 256) {
        shared_sum[tid] = histogram[tid];
    }
    __syncthreads();

    // Réduction pour calculer la somme
    for (int stride = blockDim.x / 2; stride > 0; stride /= 2) {
        if (tid < stride) {
            shared_sum[tid] += shared_sum[tid + stride];
        }
        __syncthreads();
    }

    if (tid == 0) {
        atomicAdd(&somme, shared_sum[0]);
    }
    __syncthreads();

    if (tid < 256) {
        float weight = (float)histogram[tid] / somme;
        float mean = tid * weight;

        atomicAdd(&weights[T * 2], weight);
        atomicAdd(&moyennes[T * 2], mean);

        if (T == 255) {
            caracteristiques[tid] = weights[tid] * weights[tid + 1] * powf(moyennes[tid] - moyennes[tid + 1], 2);
        }
    }
    __syncthreads();

    // Calcul du seuil
    __shared__ float max_caracteristique;
    if (tid == 0) {
        max_caracteristique = 0.0f;
    }
    __syncthreads();

    if (tid < 256) {
        atomicMax(&max_caracteristique, caracteristiques[tid]);
    }
    __syncthreads();

    // Application du seuil
    if (idx < n) {
        imageResult[idx] = (image[idx] > max_caracteristique) ? 255 : 0;
    }
}

int main() {
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    int width, height, channels;
    unsigned char *image = stbi_load("/content/images.png", &width, &height, &channels, 1);
    if (image == NULL) {
        printf("Erreur lors du chargement de l'image\n");
        return 1;
    }

    int n = width * height;
    unsigned char *h_imageResult = (unsigned char *)malloc(n * sizeof(unsigned char));
    unsigned char *d_image, *d_imageResult;
    int *d_histogram;
    float *moyennes, *weights, *caracteristiques;

    cudaMalloc((void **)&d_image, n * sizeof(unsigned char));
    cudaMalloc((void **)&d_imageResult, n * sizeof(unsigned char));
    cudaMalloc((void **)&d_histogram, 256 * sizeof(int));
    cudaMalloc((void **)&moyennes, 256 * sizeof(float));
    cudaMalloc((void **)&weights, 256 * sizeof(float));
    cudaMalloc((void **)&caracteristiques, 256 * sizeof(float));

    cudaMemcpy(d_image, image, n * sizeof(unsigned char), cudaMemcpyHostToDevice);

    int blockSize = 256;
    int numBlocks = (n + blockSize - 1) / blockSize;

    cudaEventRecord(start, 0);
    otsu_segmentation<<<numBlocks, blockSize>>>(d_image, n, d_imageResult, d_histogram, moyennes, weights, caracteristiques);
    cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop);

    float time = 0;
    cudaEventElapsedTime(&time, start, stop);

    cudaMemcpy(h_imageResult, d_imageResult, n * sizeof(unsigned char), cudaMemcpyDeviceToHost);

    printf("Temps total d'exécution : %.3f millisecondes\n", time);

    stbi_write_png("otsu_segmented_image.png", width, height, 1, h_imageResult, width);

    stbi_image_free(image);
    free(h_imageResult);
    cudaFree(d_image);
    cudaFree(d_imageResult);
    cudaFree(d_histogram);
    cudaFree(moyennes);
    cudaFree(weights);
    cudaFree(caracteristiques);

    return 0;
}
